<img src="VSM_logo.gif" width="500" align="center">

# VSM - Volcanic and Seismic source Modelling
### Version 1.0

**VSM - Volcanic and Seismic source Modelling** is a Python tool to perform inversions of geodetic data.

**Code** https://github.com/EliTras/VSM \
**License** E. Trasatti - INGV (elisa.trasatti@ingv.it), covered by GNU-GPL License https://github.com/EliTras/VSM/blob/main/license.lic

This Notebook is a guide to create the input file for VSM. The last cells run VSM.

Input
- data
- forward model(s)
- inversion settings

... Before to start let's decide
- working folder
- name of the input file to be written in that folder (and where results will be stored)

If it's the very first time, it is strongly suggested to create a dedicated environment. The libraries needed by VSM are listed in requirements.txt (numpy, pandas, matplotlib, corner, emcee, pyshp, jupyterlab, cartopy). Instructions in readme.txt.

In [ ]:
import os
import numpy as np

## Filename
Add the path of the working folder and the name of the VSM input file to be created.

In [ ]:
# path of the working folder
folder_inout = '../USECASE/mogi1NA'
filename_in = 'VSM_input_mogi.txt'

## Input data
Input data may be either displacements (m) or velocities (m/yr). If more than an input file is considered, of course the unit must be the same (all displacements or all velocities). For all datapoints, *East* and *North* are the coordinates (metric, preferred system is projected UTM).
Accepted data types are
- InSAR
- GNSS
- Levelling
- EDM
- Tilt
- Strain

### InSAR
This data refers to InSAR data, both single inferterograms or multi-temporal InSAR. Results are typycally in terms of cumulative displacements or annual velocities. The InSAR maps are usually made of thounsands of pixels so it is needed to downsample the dataset first. VSM uses downsampled InSAR datasets.
- up to 10 datasets from different sensors or missions
- a single data file is inserted as ` path/sarfile.txt `
- for multiple datasets just leave a blank between fullpath names ` path/sarfile1.txt path/sarfile2.txt `
- accepted formats are comma separated files (.csv), plain text (.txt), ESRI Shapefile (.shp). Ascii files may come with or without header in the first line
- format of columns
       East North Data Error LOS_E LOS_N LOS_Z
       
where *Data* is the LOS (Line of Sight) displacement or velocity. *LOS_E*, *LOS_N*, *LOS_Z* are the East, North and vertical components of the versor of the Line of Sight of the sensor.

### GNSS
GNSS data are typically composed of 3D displacement or mean velocity at each benchmark.
- the data file is inserted as ` path/gpsfile.txt `
- accepted formats are comma separated files (.csv), plain text (.txt), ESRI Shapefiles (.shp). Ascii files may come with or without header in the first line
- one row for each station
- format of columns
       East North Data_E Data_N Data_Z Error_E Error_N Error_Z

where *Data_E* is the East component of the displacement or velocity. Same applies to the North (N) and vertical (Z) components and to the error associated.

### Levelling
Levelling data are the difference in the elevation of benchmarks in a given time window.
- the data file is inserted as ` path/levfile.txt `
- accepted formats are comma separated files (.csv), plain text (.txt), ESRI Shapefiles (.shp). Ascii files may come with or without header in the first line
- format of columns
       East North Data Error

where *Data* is the different of elevation in the given time period.

### EDM
EDM (Electro-optical Distance Measuring) technique measures the difference in the horizontal position between two benchmarks.
- the data file is inserted as ` path/edmfile.txt `
- accepted formats are comma separated files (.csv), plain text (.txt), ESRI Shapefiles (.shp). Ascii files may come with or without header in the first line
- format of columns
       East1 North1 East2 North2 Data Error

Each row lists the coordinates of the start (1) and end (2) benchmarks. *Data* is one single value, representing the change in the distance between two benchmarks in a given time widow.

### Tilt
Tilt is a measure of the displacement gradient. It is the inclination of the vertical along a horizontal direction. The horizontal derivatives of the vertical deformation correspond to the East and North components of tilt. 
- the data file is inserted as ` path/tiltfile.txt `
- accepted formats are comma separated files (.csv), plain text (.txt), ESRI Shapefiles (.shp). Ascii files may come with or without header in the first line
- format of columns
       East North Data_E Data_N Error_E Error_N

where *Data_E* and *Data_N* are the tilt along East (E) and North (N) directions, and the associated error. If the tiltmeter is not oriented along East or North, data should be projected accordingly. Tilt is non-dimensional and the unit in VSM is microradians (ppm).

### Strain
Strain is a measure of the displacement gradient. The volumetric strain is the unit change in volume, i.e. it combines the change of the displacement in the three components. 
- the data file is inserted as ` path/strainfile.txt `
- accepted formats are comma separated files (.csv), plain text (.txt), ESRI Shapefiles (.shp). Ascii files may come with or without header in the first line
- format of columns
       East North Data Error

Strain is non-dimensional and the unit in VSM is microstrains (ppm).

In [ ]:
#Add the input files. By default all file names are 'None'. Set to 'None those unused'

# SAR filename(s)
sar_file = '../DATA/CF_2011_13/obs_sar1.txt ../DATA/CF_2011_13/obs_sar2.txt'
#GNSS filename
gps_file = '../DATA/CF_2011_13/obs_gps.txt'
# levelling filename
lev_file = 'None'
# EDM filename
edm_file = 'None'
#tilt filename
tlt_file = 'None'
# strain filename
srn_file = 'None'

### Weights
Due to the possibility to combine different datasets, the associated weights can be defined. Weights can be arbitrary. SAR data have a unique weight even if it can be composed of more than one dataset.

In [ ]:
# SAR weight
sar_weight = 1.
# GNSS weight
gps_weight = 1.
# levelling weight
lev_weight = 0.
# EDM weight
edm_weight = 0.
# tilt weight
tlt_weight = 0.
# strain weight
srn_weight = 0.

###  Start writing the VSM input text file
In this step the VSM settings about input/output and data are written in the input file as plain text. Just execute this box!

In [ ]:
f = open(os.path.join(folder_inout,filename_in), "w")
f.write(folder_inout+'\n')
f.write(sar_file+'\n'+gps_file+'\n'+lev_file+'\n'+edm_file+'\n'+tlt_file+'\n'+srn_file+'\n')
f.write(str(sar_weight)+'\n'+str(gps_weight)+'\n'+str(lev_weight)+'\n'+str(edm_weight)+'\n'+str(tlt_weight)+'\n'+str(srn_weight)+'\n');

## Forward model
VSM accepts the most common analytical forward models of deformation sources usually employed for geodetic data modelling. They can be also combined to create a forward model of up to 10 different sources whose parameters can be constrained by the VSM inversion.
Models are
- Mogi Point Source (Mogi, 1958)
- Mogi Finite Volume (Mc Tigue, 1987)
- Penny-shaped Crack (Fialko et al., 2001)
- Spheroid (Yang et al., 1988)
- Moment Tensor Source (Davis, 1986)
- Rectangular Dislocation (Okada, 1985)

Each forward source is characterized by specific parameters, whose range for the search should be determined. Coordinates and dimensions are in meters while intensity parameters are in accordance with the data employed. E.g., if data are displacements (meters), the volume variation is in m$^3$, while if ground velocities (m/yr) are used as input data, they lead to volume variation rate (m$^3$/yr). Same applies to overpressure for pressurized sources and amount of slip for the fault model.

**Additional info**

Initially, it must be defined
- elastic constants (shear modulus and Poisson ratio)
- total number of sources

In [ ]:
# shear modulus (Pa)
mu = 5e10
# Poisson ratio
ni = 0.25

# number of sources
num_sources = 1

# write this part in the input file
f.write(str(mu)+'\n')
f.write(str(ni)+'\n')
f.write(str(num_sources)+'\n');

### Definition of forward model

Define the sources with the range of search for each parameter. For each chosen source it must be reported the list of the parameters with minimum and maximum (minimum $<=$ maximum). To fix a parameter, just choose minimum and maximum equal to the desired value. 

Each forward model is identified by a number from 0 to 5. Below are reported details on parameters and a sample of instructions to copy and paste in the code cells. The depth is positive.

#### Mogi Point Source
**Model 0** The Mogi (1958) point source model represents an isotropic point source. Parameters:
- East coordinate of the source center
- North coordinate of the source center
- Depth of the source source center
- Volume variation

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`Dvol  = (0,0)` \
`bounds = [east, north, depth, Dvol] `

#### Mogi Finite Volume
**Model 1** The McTigue (1987) finite volume model is an evolution of the Mogi model and represents a spherical source with finite volume. Parameters:
- East coordinate of the source center
- North coordinate of the source center
- Depth of the source center
- Radius of the sphere
- $\Delta P/ \mu$ (Overpressure vs shear modulus ratio)

*Note* \
$\Delta P/ \mu$ is adimensional.

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`radius  = (0,0)` \
`DP_mu  = (0,0)` \
`bounds = [east, north, depth, radius, DP_mu] `

#### Penny-shaped crack
**Model 2** The penny shaped crack (Fialko et al., 2001) is a disk with a radius and no vertical extension. Parameters:
- East coordinate of the disk center
- North coordinate of the disk center
- Depth of the disk center
- Radius of the disk
- $\Delta P/ \mu$ (Overpressure vs shear modulus ratio)

*Note* \
$\Delta P/ \mu$  is adimensional.

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`radius  = (0,0)` \
`DP_mu  = (0,0)` \
`bounds = [east, north, depth, radius, DP_mu] `

#### Spheroid
**Model 3** The spheroid (Yang et al., 1988) is a finite-volume cavity with a constant overpressure on the boundary. It can be arbitrarily oriented in space Parameters:
- East coordinate of the source center
- North coordinate of the source center
- Depth of the source center
- Semi-major axis
- Ratio of semi-minor vs semi-major axes
- $\Delta P/ \mu$ (Overpressure vs shear modulus ratio)
- Strike angle
- Dip angle

*Note* \
The ratio is < 1. $\Delta P/ \mu$  is adimensional. The strike is considered according to Yang et al. (1988), i.e., degrees clockwise from East. It means if the spheroid dips to East the strike = 0°, if it dips to North the strike = 90° etc. The dip angle is 0° = horizontal 90°= vertical.

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`axis  = (0,0)` \
`ratio  = (0,0)` \
`DP_mu  = (0,0)` \
`strike  = (0,0)` \
`dip  = (0,0)` \
`bounds = [east, north, depth, axis, ratio, DP_mu, strike, dip] `

#### Moment tensor
**Model 4** The moment tensor source (Davis, 1986) is a general point source defined by a combination of dipoles and double forces, P$_{ij}$. The diagonal elements of the P$_{ij}$ matrix are the dipoles, and the off-diagonal are the double-couples according to cartersian coordinates. It may represent a sphere (three equal dipoles), a dislocation (double couple), and a subset of the possible combination of the component may represent an equivalent triaxial elliposid arbitrary oriented. The P$_{ij}$ matrix must be interpreted following the tables in Davis (1986), to find the aspect ratio of the equivalent ellipsoid and its orientation. Parameters:
- East coordinate of the source center
- North coordinate of the source center
- Depth of the source center
- P$_{xx}$
- P$_{yy}$
- P$_{zz}$
- P$_{xy}$
- P$_{yz}$
- P$_{zx}$

*Note* \
According to Davis (1986), the P$_{ij}$ matrix is expressed in Pa multiplied by volume vs shear modulus.

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`Pxx  = (0,0)` \
`Pyy  = (0,0)` \
`Pzz  = (0,0)` \
`Pxy  = (0,0)` \
`Pyz  = (0,0)` \
`Pzx  = (0,0)` \
`bounds = [east, north, depth, Pxx, Pyy, Pzz, Pxy, Pyz, Pzx] `

#### Rectangular dislocation
**Model 5** The  dislocation model (Okada, 1985) is a rectangular plane undergoing shear slip (along dip and/or strike) and/or tensile opening/closing. Two configurations are available. The first one considers strike-slip and dip-slip on the fault plane, while the other uses the total slip and the rake angle. Parameters:
- East coordinate of the top left corner
- North coordinate of the top left corner
- Depth of the top
- Length
- Width
- Strike angle
- Dip angle
- Strike-slip, or, conversely, total slip
- Dip-slip, or, conversely, rake angle
- Tensile movement

*Note* \
The strike angle is the orientation of the plane seen from the top left corner and it is measured according to Okada (1985), from North counter-clockwise, i.e., strike = 0° if the fault plane is oriented North-South and dips to East, strike = 90° if the fault is East-West trending and dips to the South, etc. The tensile movement is > 0 for opening, < 0 for closure.
Must specify also 'S' for strike and dip slip, 'R' for total slip and rake (for the generic param1 and param2).

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`length  = (0,0)` \
`width  = (0,0)` \
`strike  = (0,0)` \
`dip  = (0,0)` \
`param1  = (0,0)` \
`param2  = (0,0)` \
`opening  = (0,0)` \
`bounds = [east, north, depth, length, width, strike, dip, param1, param2, opening] `

In [ ]:
# define the source identifier of the forward model
# 0=Mogi 1=McTigue 2=Sill 3=Spheroid 4=Ellipsoid 5=Fault/Dike
# comment or uncomment the lines needed, based on the number of sources employed
sorg_identifier = 0
#parameters minimum and maximum (minimum <= maximum)
param1 = (422000, 432000)
param2 = (4515000,4525000)
param3 = (2000,6000)
param4 = (1e6, 2e7)
bounds = [param1,param2,param3,param4]

#Uncomment and define the style ('S' or 'R') in case of Okada (Fault/Dike source)
#okada_mode = 'S'

# write this part in the input file
if(sorg_identifier!= 5):
    f.write(str(sorg_identifier)+'\n')
else:
    f.write(str(sorg_identifier)+' '+okada_mode+'\n')
       
for k in range(len(bounds)):
    row = str(list(bounds[k])[0])+'\t'+str(list(bounds[k])[1])+'\n'
    f.write(row)

## VSM settings

### Inversion algorithms
There are two inversion tools included in VSM
- **NA** is the **Neighbourhood Algorithm**, a global optimizer with sampling based on the Voronoi cells theory (Sambridge, 1999). The parameters needed are:
    - number of samples at each iteration `sampl1`
    - number of re-samples from each iteration `sampl2`
    - number of iterations `sampl3`
- **BI** is the **Bayesian Inference**, based on the Bayes theory and MCMC sampling. The parameters needed are:
    - number of random walks `sampl1`
    - number of steps for each random walk `sampl2`

In [ ]:
# inversion choice NA = 0 BI = 1
inversion_choice = 0
# accordingly define the sampling parameters
sampl1 = 1000
sampl2 = 300
sampl3 = 10
f.write(str(inversion_choice)+'\n')
if(inversion_choice == 0):
    f.write(str(sampl1)+' '+str(sampl2)+'\n')
    f.write(str(sampl3)+'\n')
else:
    f.write(str(sampl1)+'\n')
    f.write(str(sampl2)+'\n')

### Plots
VSM plots the 1D and 2D parameters distributions. Also, the number of burn-in (i.e., excluded) samples can be defined, considering that at the beginning both methods are sampling low probability models also. If no plots are desired, set the number of burn-in to `-1`

In [ ]:
# Number of burn-in models. Default is 2000. No plots for -1
num_skip = 1000
f.write(str(num_skip)+'\n');

In [ ]:
# END of settings for the VSM input!
f.close()

## And now?
Now the VSM input is stored in the specified folder and ready to be used in a VSM run.
VSM may be launched by
- Jupyter Notebook, e.g., run the following cells or VSM_utilities.ipynb
- Python script, e.g., VSM_test.py
- Graphical User Interface VSM_GUI.py

Also consider VSM_utilities.py and/or VSM_utilities.ipynb for post-processing and further analysis.

In [ ]:
#import sys
#sys.path.append('path-to-VSM-folder')

import VSM

In [ ]:
# Read VSM input
VSM.read_VSM_settings(os.path.join(folder_inout,filename_in))

In [ ]:
# Run VSM
VSM.iVSM()